In [151]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import pymongo
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk
import numpy as np

In [152]:
# driver_genius = webdriver.Chrome(ChromeDriverManager().install())
# ouvrir le site et accepter les cookies
# driver_genius.get("https://genius.com/Miley-cyrus-flowers-lyrics")
# driver_genius.find_element(By.XPATH,"/html/body/div[2]/div[3]/div[1]/div/div[2]/div/button[1]").click()
# driver_genius.find_element(By.CSS_SELECTOR,"SongInfo__Title-nekw6x-1 iRKrFW").text
# for i in range(len(genius)):

#     # activer la recherche
#     element = driver_genius.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/form/input")
#     element.send_keys(genius[i][0]+' '+genius[i][1], Keys.ENTER)


#     # clicker sur la chanson
#     driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a/div[2]").click()
#     # date = driver_genius.find_element(By.XPATH,"/html/body/div[1]/main/div[1]/div[4]/div/div[1]/div[2]/div")
#     # print(date.text)
#     # print(driver_genius.find_element(By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div[1]/div[2]/div").text)
#     print(i)
#     driver_genius.get("https://genius.com/")
#     # on revient à l'accueil
#     driver_genius.back()
#     driver_genius.back()


#     WebDriverWait(driver_genius, timeout=10)
# time.sleep(100)


In [153]:
df = pd.read_csv('dataframe.csv',delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            4200 non-null   object
 1   Artist           4200 non-null   object
 2   Rank             4200 non-null   int64 
 3   Last Week        4200 non-null   object
 4   Peak Positon     4200 non-null   int64 
 5   Weeks on charts  4200 non-null   int64 
 6   Date             4200 non-null   object
dtypes: int64(3), object(4)
memory usage: 229.8+ KB


In [154]:
df['Last Week'] = df['Last Week'].replace("-",0)
df['Last Week']

0         0
1         4
2         2
3         1
4         3
       ... 
4195    193
4196    174
4197      0
4198    181
4199      0
Name: Last Week, Length: 4200, dtype: object

In [155]:
df['Last Week']= df['Last Week'].astype('int64')
df['Date']=pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Title            4200 non-null   object        
 1   Artist           4200 non-null   object        
 2   Rank             4200 non-null   int64         
 3   Last Week        4200 non-null   int64         
 4   Peak Positon     4200 non-null   int64         
 5   Weeks on charts  4200 non-null   int64         
 6   Date             4200 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 229.8+ KB


In [156]:
data = df.to_dict('records')
# data = df.to_json(date_format='dd/mm/yyyy',orient='records')
# data

In [157]:
# client = MongoClient("mongodb://localhost:27017/")
# db = client["mydatabase"]
# collection = db["mycollection"]

# print(client.list_database_names())
# print(db.list_collection_names())
# client.drop_database(db)
# collection.drop()
# print(client.list_database_names())
# print(db.list_collection_names())

# db = client["mydatabase"]
# collection = db["mycollection"]
# print(type(client))
# print(type(db))
# print(collection)

# collection.insert_many(data)
# collection.find_one()

# cursor = collection.find()
# # next(cursor)
# for document in cursor :
#     # print('-----')
#     print(document)

In [158]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,"scheme": "http"}])

In [159]:
es.ping()

True

In [160]:
# document = {"Title":"Anti-Hero","Artist":"Taylor Swift","Rank":1,"Last Week":"2","Peak Positon":1,"Weeks on charts":4,"Date":"26\\/11\\/2022"}

In [161]:
# response = es.index(index="billboard", id=1 ,document=document)
# response['result']
# response

In [162]:
# res = es.get(index="billboard",id=1)
# # pprint.pprint(res["_source"])
# res["_source"]

In [163]:
# # define the index name and document type
# index_name = 'billboard'
# doc_type = '_doc'

# # define the action list to be passed to the bulk method
# actions = [
#     {
#         '_index': index_name,
#         '_type': doc_type,
#         '_source': data
#     }
#     for i in data
# ]

# # use the bulk method to index all documents
# response = bulk(es, actions=actions)

In [164]:
es.indices.exists('billboard')

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\1081182445.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.exists('billboard')


True

In [165]:
def generate_data(data):
    for docu in data:
        yield {
            "_index": "billboard",
            "_type": "song",
            "_source": {k:v if v else None for k,v in docu.items()},
        }
        
if es.indices.exists('billboard')==True:
    es.indices.delete(index='billboard')
    bulk(es, generate_data(data))
else :
    bulk(es, generate_data(data))

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\2795846709.py:9: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists('billboard')==True:
c:\Users\camil\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [166]:
es.indices.exists('billboard')

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\1081182445.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.exists('billboard')


True

In [167]:
result = es.search(index="billboard", query= {"match_all": {}})
result

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [168]:
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

'0 document correspondent à la requêtes qui a pris 0 ms'

In [169]:
!curl http://localhost:9200/_cat/indices?v #!curl http://elasticsearch:9200/_cat/indices?v

health status index                    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .kibana_task_manager_1   EXIwN7TRSVqNquk2ayLb4A   1   0          2            1     23.9kb         23.9kb
green  open   .apm-agent-configuration UL5dxL77QXCnt2-BKIT2aQ   1   0          0            0       283b           283b
yellow open   billboard                LZUeTCY5SKSpDT_vNeLexg   1   1       4200            0    555.4kb        555.4kb
green  open   .kibana_1                jodusnl1QCuev0azrBTcxg   1   0         21            1       26kb           26kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   600  100   600    0     0  23205      0 --:--:-- --:--:-- --:--:-- 24000
curl: (3) URL using bad/illegal format or missing URL
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: elasticsearch


In [170]:
# cherche toutes les chansons avec pour artiste BLACKPINK
QUERY = {
    "query": {
        "term": {
            "Artist.keyword": "BLACKPINK"
        }
    }
}

In [171]:
result = es.search(index="billboard", body=QUERY)
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\3680423717.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'51 documents correspondent à la requêtes qui a pris 48 ms'

In [172]:
# affiche le titre et le rang pour le resultat de la requete
{elt['_source']['Title']:elt['_source']['Rank'] for elt in result["hits"]["hits"]}

{'Shut Down': 4,
 'Pink Venom': 10,
 'Typa Girl': 16,
 'Hard To Love': 27,
 'The Happiest Girl': 34,
 'Yeah Yeah Yeah': 43,
 'Tally': 52,
 'Ready For Love': 60}

In [173]:
# result['hits']['hits']
QUERY = {
    "query": {
        "term": {
            "Title": "Christmas"
        }
    }
}
result = es.search(index="billboard", body=QUERY)
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\1732994941.py:9: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'0 document correspondent à la requêtes qui a pris 0 ms'

In [194]:
# affiche les top 10 d'artistes qui ont le plus de chansons dans le billboard
QUERY ={
  "aggs": {
    "artist": {
      "terms": {
        "field": "Artist.keyword",
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["artist"]["buckets"]

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\431253462.py:14: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key': 'Taylor Swift', 'doc_count': 150},
 {'key': 'Bad Bunny', 'doc_count': 102},
 {'key': 'Ed Sheeran', 'doc_count': 95},
 {'key': 'SZA', 'doc_count': 89},
 {'key': 'The Weeknd', 'doc_count': 79},
 {'key': 'Eminem', 'doc_count': 72},
 {'key': 'Harry Styles', 'doc_count': 59},
 {'key': 'Arctic Monkeys', 'doc_count': 56},
 {'key': 'BLACKPINK', 'doc_count': 51},
 {'key': 'Drake & 21 Savage', 'doc_count': 51}]

In [ ]:
{
  "aggs": {
    "genres": {
      "terms": {
        "field": "genre.keyword"
      },
      "aggs": {
        "avg_rating": {
          "avg": {
            "field": "rating"
          }
        },
        "sales": {
          "sum": {
            "field": "sales"
          }
        }
      }
    }
  }
}


In [198]:
# chaque mois top des chansons qui apparaissent le plus sur le billbaord (pas en fonction de leur rang)
QUERY = {
  "aggs": {
    "song_by_month": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "month"
      },
      "aggs": {
        "title": {
          "terms": {
            "field": "Title.keyword"
          }
        }
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["song_by_month"]["buckets"]

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\3956439160.py:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key_as_string': '2022-10-01T00:00:00.000Z',
  'key': 1664582400000,
  'doc_count': 1000,
  'title': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 950,
   'buckets': [{'key': '2 Be Loved (Am I Ready)', 'doc_count': 5},
    {'key': '505', 'doc_count': 5},
    {'key': 'About Damn Time', 'doc_count': 5},
    {'key': 'After Like', 'doc_count': 5},
    {'key': "All Too Well (Taylor's Version)", 'doc_count': 5},
    {'key': 'Another Love', 'doc_count': 5},
    {'key': 'As It Was', 'doc_count': 5},
    {'key': 'Atlantis', 'doc_count': 5},
    {'key': 'B.O.T.A. (Baddest Of Them All)', 'doc_count': 5},
    {'key': 'Baby Shark', 'doc_count': 5}]}},
 {'key_as_string': '2022-11-01T00:00:00.000Z',
  'key': 1667260800000,
  'doc_count': 800,
  'title': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 755,
   'buckets': [{'key': 'Miss You', 'doc_count': 7},
    {'key': 'Die For You', 'doc_count': 6},
    {'key': '505', 'doc_count': 4},
    {'key': 'About Damn Time', 'doc_

In [175]:
QUERY= {
  "aggs": {
    "avg_weeks_on_chart": {
      "avg": {
        "field": "Weeks on charts"
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["avg_weeks_on_chart"]

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\1238742633.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


{'value': 31.349047619047617}

In [176]:
QUERY = {
  "aggs": {
    "top_song_per_month": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "month"
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["top_song_per_month"]

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\2806668549.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


{'buckets': [{'key_as_string': '2022-10-01T00:00:00.000Z',
   'key': 1664582400000,
   'doc_count': 1000},
  {'key_as_string': '2022-11-01T00:00:00.000Z',
   'key': 1667260800000,
   'doc_count': 800},
  {'key_as_string': '2022-12-01T00:00:00.000Z',
   'key': 1669852800000,
   'doc_count': 1000},
  {'key_as_string': '2023-01-01T00:00:00.000Z',
   'key': 1672531200000,
   'doc_count': 1000},
  {'key_as_string': '2023-02-01T00:00:00.000Z',
   'key': 1675209600000,
   'doc_count': 400}]}

In [177]:
# QUERY ={
#   "aggs": {
#     "title": {
#       "terms": {
#         "field": "Title.keyword"
#       },
#       "aggs": {
#         "artists": {
#           "terms": {
#             "field": "Artist.keyword"
#           }
#         }
#       }
#     }
#   }
# }
QUERY={
  "aggs": {
    "title": {
      "terms": {
        "field": "Title.keyword"
      },
      "aggs": {
        "artists": {
          "terms": {
            "field": "Artist.keyword"
          },
          "aggs": {
            "unique_artists": {
              "cardinality": {
                "field": "Artist.keyword"
              }
            }
          }
        }
      }
    }
  }
}

result = es.search(index="billboard", body=QUERY)
result = result["aggregations"]["title"]["buckets"]
result

C:\Users\camil\AppData\Local\Temp\ipykernel_8184\2297239962.py:41: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key': 'Die For You',
  'doc_count': 27,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'The Weeknd',
     'doc_count': 21,
     'unique_artists': {'value': 1}},
    {'key': 'Joji', 'doc_count': 6, 'unique_artists': {'value': 1}}]}},
 {'key': 'Miss You',
  'doc_count': 22,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Oliver Tree & Robin Schulz',
     'doc_count': 18,
     'unique_artists': {'value': 1}},
    {'key': 'southstar', 'doc_count': 4, 'unique_artists': {'value': 1}}]}},
 {'key': 'About Damn Time',
  'doc_count': 21,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Lizzo',
     'doc_count': 21,
     'unique_artists': {'value': 1}}]}},
 {'key': 'Another Love',
  'doc_count': 21,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Tom Odell',
     'doc_count': 21,
     '

In [178]:
for i in range(len(result)):
  title = result[i]['key'],result[i]['doc_count']
  print(title)
for i in range(len(result)):
  artistesdict = result[i]['artists']['buckets']
  for j in range(len(artistesdict)):
    artiste = artistesdict[j]['key'],artistesdict[j]['doc_count']
    print(artiste)

('Die For You', 27)
('Miss You', 22)
('About Damn Time', 21)
('Another Love', 21)
('As It Was', 21)
('Baby Shark', 21)
('Bad Habit', 21)
('Bad Habits', 21)
('Believer', 21)
('Blinding Lights', 21)
('The Weeknd', 21)
('Joji', 6)
('Oliver Tree & Robin Schulz', 18)
('southstar', 4)
('Lizzo', 21)
('Tom Odell', 21)
('Harry Styles', 21)
('Pinkfong', 21)
('Steve Lacy', 21)
('Ed Sheeran', 21)
('Imagine Dragons', 21)
('The Weeknd', 21)


In [179]:
# Define the Elasticsearch document
class MyDocument(Document):
    title = Text()
    artist = Text()
    rank = Integer()
    class Meta:
        index = "your_index"


In [180]:
# # Transfer the data from MongoDB to Elasticsearch
# for document in collection.find():
#     doc = MyDocument(meta={"id": document["_id"]}, title=document["Title"], artist=document["Artist"],rank=document["Rank"])
#     doc.save(using=es)